# Histogram-Based Approach

In [18]:
import numpy as np
from matplotlib import pyplot as plt
from utils.edge_extraction import extract_working_region
import cv2
from utils.image_conversion import *

In [19]:
image_green1 = cv2.imread("./data/5.34.1.png", cv2.IMREAD_UNCHANGED)
image_green2 = cv2.imread("./data/5.38.15.png", cv2.IMREAD_UNCHANGED)
image_yellow = cv2.imread("./data/5.34.34.png", cv2.IMREAD_UNCHANGED)

working_region_image_green1 = extract_working_region(image_green1, threshold=10)
working_region_image_green2 = extract_working_region(image_green2, threshold=10)
working_region_image_yellow = extract_working_region(image_yellow, threshold=10)

fig, axes = plt.subplots(3, 2, figsize=(15, 5))
images = [image_green1, working_region_image_green1, image_green2, working_region_image_green2, image_yellow, working_region_image_yellow]
titles = ["Original Image", "Working region"]

for i, ax_row in enumerate(axes):
    for j, ax in enumerate(ax_row):
        ax.imshow(images[i * 2 + j])
        ax.set_title(titles[j])

plt.tight_layout()
plt.show()

[ WARN:0@90.337] global loadsave.cpp:248 findDecoder imread_('./data/5.34.1.png'): can't open/read file: check file path/integrity
[ WARN:0@90.337] global loadsave.cpp:248 findDecoder imread_('./data/5.38.15.png'): can't open/read file: check file path/integrity
[ WARN:0@90.338] global loadsave.cpp:248 findDecoder imread_('./data/5.34.34.png'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
def get_only_valid_pixels(input_image: np.ndarray) -> np.ndarray:
    # Extract pixels with alpha channel value different from 0
    valid_pixels = input_image[input_image[:, :, 3] != 0]

    # Reshape to (num_pixels, 4)
    reshaped_image = valid_pixels.reshape((-1, 4))

    # Determine the dimensions of the reshaped image
    num_pixels = reshaped_image.shape[0]
    width = int(np.sqrt(num_pixels))
    height = (num_pixels + width - 1) // width

    # Pad the reshaped image to make it rectangular
    padded_image = np.zeros((height * width, 4), dtype=np.uint8)
    padded_image[:num_pixels, :] = reshaped_image

    # Reshape the padded image to the desired dimensions
    final_image = padded_image.reshape((height, width, 4))

    # Split channel
    b, g, r, a = cv2.split(final_image)
    rgb_image = cv2.merge((b, g, r))

    return rgb_image

In [ ]:
working_region_image_green1 = get_only_valid_pixels(working_region_image_green1)
working_region_image_green2 = get_only_valid_pixels(working_region_image_green2)
working_region_image_yellow = get_only_valid_pixels(working_region_image_yellow)

working_region_image_green1 = cv.cvtColor(working_region_image_green1, cv.COLOR_BGR2HSV)
working_region_image_green2 = cv.cvtColor(working_region_image_green2, cv.COLOR_BGR2HSV)
working_region_image_yellow = cv.cvtColor(working_region_image_yellow, cv.COLOR_BGR2HSV)


fig, axes = plt.subplots(1, 3, figsize=(15, 5))
images = [working_region_image_green1, working_region_image_green2,working_region_image_yellow]
titles = ["Working region"]

for i, ax in enumerate(axes):
    ax.imshow(images[i])
    ax.set_title(titles[0])

plt.show()

In [ ]:
hist_img_green1 = cv2.calcHist([working_region_image_green1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
cv2.normalize(hist_img_green1, hist_img_green1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

hist_img_green2 = cv2.calcHist([working_region_image_green2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
cv2.normalize(hist_img_green2, hist_img_green2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

hist_img_yellow = cv2.calcHist([working_region_image_yellow], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
cv2.normalize(hist_img_yellow, hist_img_yellow, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

cp_green_green = cv2.compareHist(hist_img_green1, hist_img_green2, cv2.HISTCMP_CORREL)
print(f"Similarity Score (Green-Green): ", round(cp_green_green, 2))

cp_green_yellow = cv2.compareHist(hist_img_green1, hist_img_yellow, cv2.HISTCMP_CORREL)
print(f"Similarity Score (Green1-Yellow): ", round(cp_green_yellow, 2))

cp_green2_yellow = cv2.compareHist(hist_img_green2, hist_img_yellow, cv2.HISTCMP_CORREL)
print(f"Similarity Score (Green2-Yellow): ", round(cp_green2_yellow, 2))

cp_yellow_yellow = cv2.compareHist(hist_img_yellow, hist_img_yellow, cv2.HISTCMP_CORREL)
print(f"Similarity Score (Yellow-Yellow): ", round(cp_yellow_yellow, 2))

In [ ]:
hist_img_green1

In [20]:
chans = cv2.split(working_region_image_yellow)
colors = ("b", "g", "r")
plt.figure()
plt.title("'Flattened' Color Histogram")
plt.xlabel("Bins")
plt.ylabel("# of Pixels")
# loop over the image channels
for (chan, color) in zip(chans, colors):
	# create a histogram for the current channel and plot it
	hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
	plt.plot(hist, color=color)
	plt.xlim([0, 256])

NameError: name 'working_region_image_yellow' is not defined

In [ ]:
hist = cv2.calcHist([working_region_image_green1], [0, 1, 2],
	None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
print("3D histogram shape: {}, with {} values".format(
	hist.shape, hist.flatten().shape[0]))

In [21]:
hist = cv2.calcHist([working_region_image_yellow], [0, 1, 2],
	None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
print("3D histogram shape: {}, with {} values".format(
	hist.shape, hist.flatten().shape[0]))

NameError: name 'working_region_image_yellow' is not defined

In [22]:
def plot_histogram(image, title, mask=None):
    # split the image into its respective channels, then initialize
    # the tuple of channel names along with our figure for plotting
    chans = cv2.split(image)
    colors = ("b", "g", "r")
    plt.figure()
    plt.title(title)
    plt.xlabel("Bins")
    plt.ylabel("# of Pixels")
    plt.ylim([0, 256])
    # loop over the image channels
    for (chan, color) in zip(chans, colors):
        # create a histogram for the current channel and plot it
        hist = cv2.calcHist([chan], [0], mask, [256], [0, 256])
        plt.plot(hist, color=color)
        plt.xlim([0, 256])

In [23]:
plot_histogram(working_region_image_yellow, "AAA")

NameError: name 'working_region_image_yellow' is not defined

In [ ]:
plot_histogram(working_region_image_green1, "AAA")

In [ ]:
plot_histogram(working_region_image_green2, "AAA")

In [25]:
import os
from utils.edge_extraction import *
import cv2

def create_dataset(img_dir):

    working_regions = []
    for file in os.listdir(img_dir):
        if os.path.splitext(file)[1] != ".png":
            continue

        image = cv.imread(os.path.join(img_dir, file), cv.IMREAD_UNCHANGED)
        working_region = extract_working_region(image, threshold=5)
        working_regions.append(filter_working_region(working_region)) # bgr working region

    histograms = []
    for working_region in working_regions:
        # working_region = cv2.cvtColor(working_region, cv.COLOR_BGR2HSV)
        hist_src = cv2.calcHist([working_region], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        hist_dst = cv2.normalize(hist_src, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        histograms.append(hist_dst)
        
    # calculate matrix similarity
    similarity_matrix = np.zeros((len(histograms), len(histograms)))  # Initialize similarity matrix
    for i, hist_dst in enumerate(histograms):
        for j, hist_src in enumerate(histograms):
            similarity_matrix[i, j] = 1 - cv2.compareHist(hist_dst, hist_src, cv2.HISTCMP_CORREL)

    return similarity_matrix

In [26]:
X = create_dataset("./data")
X.shape

(1036, 1036)

In [29]:
X

array([[0.        , 1.05567758, 0.5619909 , ..., 1.11709529, 1.09582639,
        1.11597879],
       [1.05567758, 0.        , 1.03626672, ..., 0.86804042, 0.7431758 ,
        0.93111592],
       [0.5619909 , 1.03626672, 0.        , ..., 1.10509346, 1.09300357,
        1.10477343],
       ...,
       [1.11709529, 0.86804042, 1.10509346, ..., 0.        , 0.9864973 ,
        0.97544006],
       [1.09582639, 0.7431758 , 1.09300357, ..., 0.9864973 , 0.        ,
        0.47693957],
       [1.11597879, 0.93111592, 1.10477343, ..., 0.97544006, 0.47693957,
        0.        ]])

In [27]:
# def histogram_similarity(hist1, hist2):
#     hist1 = hist1.reshape((8,8,8))
#     hist2 = hist2.reshape((8,8,8))
#     return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.2, min_samples=10, metric="precomputed").fit(X)
labels = db.labels_
print(labels)

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)
labels.shape

[-1 -1  0 ...  3  0 -1]
Estimated number of clusters: 6
Estimated number of noise points: 375


(1036,)

In [28]:
import shutil 

os.makedirs("clusters", exist_ok=True)

images = []
for file in os.listdir("./data"):
        if os.path.splitext(file)[1] != ".png":
            continue

        image = os.path.join("./data", file)
        images.append(image)

# Iterate over each image and corresponding label
for idx, label in enumerate(labels):

    if label == -1:
        label = "unclustered"

    # Create subfolder for each cluster
    cluster_dir = os.path.join("clusters", f"cluster_{label}")
    os.makedirs(cluster_dir, exist_ok=True)

    # Copy or move images to the corresponding cluster subfolder
    image_path = images[idx]
    image_name = os.path.basename(image_path)
    new_image_path = os.path.join(cluster_dir, image_name)
    shutil.copy(image_path, new_image_path)

    print(f"Moving {image_path} to {new_image_path}")

Moving ./data/5.26.42.png to clusters/cluster_unclustered/5.26.42.png
Moving ./data/5.26.7.png to clusters/cluster_unclustered/5.26.7.png
Moving ./data/5.14.33.png to clusters/cluster_0/5.14.33.png
Moving ./data/5.1.5.png to clusters/cluster_unclustered/5.1.5.png
Moving ./data/5.14.27.png to clusters/cluster_0/5.14.27.png
Moving ./data/5.13.12.png to clusters/cluster_unclustered/5.13.12.png
Moving ./data/5.22.26.png to clusters/cluster_0/5.22.26.png
Moving ./data/5.24.5.png to clusters/cluster_0/5.24.5.png
Moving ./data/5.22.32.png to clusters/cluster_2/5.22.32.png
Moving ./data/5.25.13.png to clusters/cluster_0/5.25.13.png
Moving ./data/5.7.27.png to clusters/cluster_0/5.7.27.png
Moving ./data/5.7.33.png to clusters/cluster_0/5.7.33.png
Moving ./data/5.3.7.png to clusters/cluster_unclustered/5.3.7.png
Moving ./data/5.9.36.png to clusters/cluster_1/5.9.36.png
Moving ./data/5.9.22.png to clusters/cluster_unclustered/5.9.22.png
Moving ./data/5.22.3.png to clusters/cluster_0/5.22.3.png
Mo